# Implementing Iteration

## Agenda

1. Review: Iteration
2. Details: *iterables*, *iterators*, `iter`, and `next`
3. Implementing iterators with classes
4. Implementing iterators with *generators* and `yield`

## 1. Review: Iteration

*Iteration* simply refers to the process of accessing — one by one — the items stored in some container. The order of the items, and whether or not the iteration is comprehensive, depends on the container.

In Python, we typically perform iteration using the `for` loop.

In [ ]:
# e.g., iterating over a list
l = [2**x for x in range(10)]
for n in l:
    print(n)

In [ ]:
# e.g., iterating over the key-value pairs in a dictionary
d = {x:2**x for x in range(10)}
for k,v in d.items():
    print(k, '=>', v)

## 2. Review: *iterables*, *iterators*, `iter`, and `next`

We can iterate over anything that is *iterable*. Intuitively, if something can be used as the source of items in a `for` loop, it is iterable.

But how does a `for` loop really work? (Review time!)

In [ ]:
l = [2**x for x in range(10)]

## 3. Implementing iterators with classes

In [ ]:
class MyIterator:
    def __init__(self, max):
        self.max = max
        self.curr = 0
        
    # the following methods are required for iterator objects
    
    def __next__(self):
        pass
    
    def __iter__(self):
        pass

In [ ]:
it = MyIterator(10)

In [ ]:
next(it)

In [ ]:
it = MyIterator(10)
while True:
    try:
        print(next(it))
    except StopIteration:
        break

In [ ]:
it = MyIterator(10)
for i in it:
    print(i)

An iterator is a *one time use object*! I.e., once we've used it to iterate over elements we cannot typically reset or "rewind" iteration. Iterable objects that can be traversed repeatedly return fresh iterators for each traversal.

In [ ]:
l = ['a', 'b', 'c', 'd', 'e']
for _ in range(3):
    for x in l:
        print(x, end=' ')

In [ ]:
l = ['a', 'b', 'c', 'd', 'e']
for _ in range(3):
    it = iter(l) # we obtain and "use up" an iterator each loop!
    while True:
        try:
            x = next(it)
            print(x, end=' ')
        except StopIteration:
            break

For a container type, we need to implement an `__iter__` method that returns an iterator.

In [ ]:
class ArrayList:
    def __init__(self):
        self.data = []
        
    def append(self, val):
        self.data.append(None)
        self.data[len(self.data)-1] = val
        
    def __iter__(self):
        pass

In [ ]:
l = ArrayList()
for x in range(10):
    l.append(2**x)

In [ ]:
it = iter(l)

In [ ]:
type(it)

In [ ]:
next(it)

In [ ]:
for x in l:
    print(x)

## 4. Implementing iterators with generators

What's a "generator"?

In [ ]:
l = [2*x for x in range(10)]
g = (2*x for x in range(10))

In [ ]:
type(l), type(g)

In [ ]:
for x in l:
    print(x)

In [ ]:
for x in g:
    print(x)

In [ ]:
dir(g)

In [ ]:
%timeit -n 1000 [2*x for x in range(10_000)]

In [ ]:
%timeit -n 1000 (2*x for x in range(10_000))

In [ ]:
g = (2*x for x in range(10_000))

In [ ]:
g[100]

In [ ]:
g[:100]

In [ ]:
sum(g)

In [ ]:
sum(g)

### Creating generator functions: `yield`

In [ ]:
def foo():
    yield

In [ ]:
foo()

In [ ]:
type(foo())

In [ ]:
def foo():
    print('hello!')
    yield
    print('goodbye!')

In [ ]:
foo()

In [ ]:
g = foo()

In [ ]:
next(g)

In [ ]:
def foo():
    yield 1
    yield 2
    yield 3

In [ ]:
g = foo()

In [ ]:
next(g)

In [ ]:
def countdown(n):
    pass

In [ ]:
for x in countdown(5):
    print(x)

In [ ]:
list(countdown(10))

### Generators as Iterators

In [ ]:
class ArrayList:
    def __init__(self):
        self.data = []
        
    def append(self, val):
        self.data.append(None)
        self.data[len(self.data)-1] = val
        
    def __iter__(self):
        pass

In [ ]:
l = ArrayList()
for x in range(10):
    l.append(2**x)

In [ ]:
for x in l:
    print(x)

In [ ]:
class ArrayList(ArrayList):
    def __repr__(self):
        return '[' + ? + ']'

In [ ]:
l = ArrayList()
for x in range(10):
    l.append(2**x)
l